In [1]:
using PorousMaterials
using DataFrames
using CSV

In [2]:
# # location of origional crystal structures used for lammps sims
# @eval PorousMaterials PATH_TO_CRYSTALS = joinpath("../mof_construction/NiPyC2_relax_sc211/")
# crystal = Crystal("NiPyC2_relax_sc211_meta_functionalized_CH2-CH2-CH3.cif")

In [3]:
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(),"../data/crystals")
# exp_crystal = Crystal("NiPyC2_experiment.cif")

In [4]:
# structurename = joinpath("CH2-CH3", "NiPyC2_relax_sc211_meta_functionalized_CH2-CH3")
# xtal_csv = CSV.read(structurename * ".min.csv")

**NOTE:** Helpful information about how "BOX" parameters are stored in the LAMMPS `*_restart.lammpstr` file can be found [here](https://lammps.sandia.gov/threads/msg10124.html)

https://lammps.sandia.gov/doc/Howto_triclinic.html 

https://lammps.sandia.gov/threads/msg10114.html

In [5]:
function lammps_output_to_crystal(structurename::String, box::Union{Box, Nothing}=nothing)
    filename = structurename * "_restart.lammpstrj"

    lines = readlines(filename)
    
    csv_filename = structurename * ".min.csv"
    xtal_csv = CSV.read(csv_filename)

    ###
    #   get the Box
    ###
    if isnothing(box)
        println(" Getting xtal box from lammps sim. ")
#         # ITEM: BOX BOUNDS xy xz yz
#         # xlo_bound xhi_bound xy
#         # ylo_bound yhi_bound xz
#         # zlo_bound zhi_bound yz
#         xlo_bound = parse(Float64, split(lines[6])[1])
#         xhi_bound = parse(Float64, split(lines[6])[2])
#         xy        = parse(Float64, split(lines[6])[3])

#         ylo_bound = parse(Float64, split(lines[7])[1])
#         yhi_bound = parse(Float64, split(lines[7])[2])
#         xz        = parse(Float64, split(lines[7])[3])

#         zlo_bound = parse(Float64, split(lines[8])[1])
#         zhi_bound = parse(Float64, split(lines[8])[2])
#         yz        = parse(Float64, split(lines[8])[3])
        
#         # xlo_bound = xlo + MIN(0.0,xy,xz,xy+xz)
#         # xhi_bound = xhi + MAX(0.0,xy,xz,xy+xz)
#         # ylo_bound = ylo + MIN(0.0,yz)
#         # yhi_bound = yhi + MAX(0.0,yz)
#         # zlo_bound = zlo
#         # zhi_bound = zhi
#         xlo = xlo_bound - min(0.0, xy, xz, xy+xz)
#         xhi = xhi_bound - max(0.0, xy, xz, xy+xz)
#         ylo = ylo_bound - min(0.0, yz)
#         yhi = yhi_bound - max(0.0, yz)
#         zlo = zlo_bound
#         zhi = zhi_bound

#         #>>> a = (xhi-xlo, 0, 0)
#         #>>> b = (xy, yhi-ylo, 0)
#         #>>> c = (xz, yz, zhi-zlo)
#         a = [xhi - xlo, 0, 0]
#         b = [xy, yhi-ylo, 0]
#         c = [xz, yz, zhi-zlo]

#         f_to_c = [a b c]
#         box = Box(f_to_c)
        
        # check that box dims are constant at each output
        @assert all(xtal_csv[:, :vol] .== xtal_csv[1, :vol])
        @assert all(xtal_csv[:, :lx]  .== xtal_csv[1, :lx])
        @assert all(xtal_csv[:, :ly]  .== xtal_csv[1, :ly])
        @assert all(xtal_csv[:, :lz]  .== xtal_csv[1, :lz])
        @assert all(xtal_csv[:, :xy]  .== xtal_csv[1, :xy])
        @assert all(xtal_csv[:, :xz]  .== xtal_csv[1, :xz])
        @assert all(xtal_csv[:, :yz]  .== xtal_csv[1, :yz])

        # assign lammps box variables
        lx = xtal_csv[end , :lx]
        ly = xtal_csv[end , :ly]
        lz = xtal_csv[end , :lz]
        xy = xtal_csv[end , :xy]
        xz = xtal_csv[end , :xz]
        yz = xtal_csv[end , :yz]
        
        # calculate lattice params
        a = lx
        b = sqrt(ly^2 + xy^2)
        c = sqrt(lz^2 + xz^2 + yz^2) 
        
        α = acos((xy*xz + ly*yz) / (b*c))
        β = acos(xz / c)
        γ = acos(xy / b)
        
        # generate box
        box = Box(a, b, c, α, β, γ)
    end

    ###
    #  get number of atoms
    ###
    n_atoms = parse(Int, lines[4])

    ###
    #  get atom coords from xyz file
    #   (not from LAMMPS output file b/c types are opaque)
    ###
    xyz_filename = structurename * "_mov.xyz"
    run(pipeline(`tail -$(n_atoms+2) $xyz_filename`, "temp.xyz"))
    atoms = read_xyz("temp.xyz")
    rm("temp.xyz")
    # x = zeros(3, n_atoms)
    # for i = 1:n_atoms
    #     x_i = split.(lines[9+i])[3:end]
    #     for k = 1:3
    #         x[k, i] = parse(Float64, x_i[k])
    #     end
    # end

    ###
    #  construct Crystal
    ###
    # Crystal(name::String, box::Box, atoms::Atoms{Frac}, charges::Charges{Frac})
    return Crystal(structurename * "_lammps", box, Frac(atoms, box), Charges{Frac}(0))
end

lammps_output_to_crystal (generic function with 2 methods)

In [6]:
# structure = "NiPyC2_relax_sc211_meta_functionalized_CH2-CH3"
# file = joinpath("CH2-CH3",structure)
# xtal = lammps_output_to_crystal(file)
# xtal.box

In [7]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the
# # :C_aro_R atom on the ring segment of the Fragment
fragments_with_overlap = ["C-3CH3", "CH2-CH2-CH3", "CH2-CH3","CH2-NH2",
                          "CH-CH2", "N-2CH3", "O-CH2-CH2-CH3",
                          "O-CH2-CH3", "O-CH3", "P-2CH3", "PH2"]

# fragments_with_overlap = ["CH2-NH2"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
substitution_types = ["meta", "ortho"]

2-element Array{String,1}:
 "meta"
 "ortho"

In [8]:
for fragment in fragments_with_overlap
    # move into dir containing lammps sims of fragment
    cd(fragment)
    println(pwd())
    for sub_type in substitution_types
        structurename = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * fragment
        xtal = lammps_output_to_crystal(structurename) # exp_crystal.box
        # check for overlapping atoms before wrapping to unit cell
        overlap_flag, overlap_pairs = overlap(xtal)
        if overlap_flag
            @warn structurename * " has overlapping atoms before wrapped to unit cell"
        end
        # make sure all of the atoms are inside the box 
        wrap!(xtal)        
        # check for overlapping atoms after wrapping to unit cell
        overlap_flag, overlap_pairs = overlap(xtal)
        if overlap_flag
            @warn structurename * " has overlapping atoms when wrapped to unit cell"
        end
        # write cif and xyz files
        write_cif(xtal, structurename * "_lammps.cif")
        strip_numbers_from_atom_labels!(xtal)
        write_xyz(xtal)
        println(xtal.name)
    end
    # move back into parent dir
    cd("..")
    @assert pwd() == "/home/ng/DTRA/lammps_sims"
end    

/home/ng/DTRA/lammps_sims/C-3CH3
 Getting xtal box from lammps sim. 


┌ Warning: NiPyC2_relax_sc211_meta_functionalized_C-3CH3 has overlapping atoms before wrapped to unit cell
└ @ Main In[8]:11
┌ Warning: NiPyC2_relax_sc211_meta_functionalized_C-3CH3 has overlapping atoms when wrapped to unit cell
└ @ Main In[8]:18


NiPyC2_relax_sc211_meta_functionalized_C-3CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_C-3CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-CH2-CH3
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_CH2-CH2-CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_CH2-CH2-CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-CH3
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_CH2-CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_CH2-CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-NH2
 Getting xtal box from lammps sim. 


┌ Warning: NiPyC2_relax_sc211_ortho_functionalized_C-3CH3 has overlapping atoms before wrapped to unit cell
└ @ Main In[8]:11
┌ Warning: NiPyC2_relax_sc211_ortho_functionalized_C-3CH3 has overlapping atoms when wrapped to unit cell
└ @ Main In[8]:18


NiPyC2_relax_sc211_meta_functionalized_CH2-NH2_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_CH2-NH2_lammps
/home/ng/DTRA/lammps_sims/CH-CH2
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_CH-CH2_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_CH-CH2_lammps
/home/ng/DTRA/lammps_sims/N-2CH3
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_N-2CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_N-2CH3_lammps
/home/ng/DTRA/lammps_sims/O-CH2-CH2-CH3
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_O-CH2-CH2-CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functionalized_O-CH2-CH2-CH3_lammps
/home/ng/DTRA/lammps_sims/O-CH2-CH3
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_meta_functionalized_O-CH2-CH3_lammps
 Getting xtal box from lammps sim. 
NiPyC2_relax_sc211_ortho_functiona

┌ Warning: NiPyC2_relax_sc211_ortho_functionalized_P-2CH3 has overlapping atoms before wrapped to unit cell
└ @ Main In[8]:11
┌ Warning: NiPyC2_relax_sc211_ortho_functionalized_P-2CH3 has overlapping atoms when wrapped to unit cell
└ @ Main In[8]:18


In [9]:
# xtal.box .== crystal.box.f_to_c